sewing machine, refrigerator, typewriter, Bookcase, Frame, lumber, TV, Audio, fax machine, chair, dish dryer, bicycle, arcade machine, TV stand, audio cabinet, Wardrobe, Display cabinet, Cupboard, Rice storage container, Stroller, Signboard, Washing machine, Speaker, Desk, Vanity table, Wall clock, Earthenware jar, Water tank, Air conditioner, Fan, Dining table, Small cabinet, Mannequin, Sink, Bedding items, Gas oven range, Heater, Cabinet, Drawer unit, Clothes drying rack, Toilet bowl, Spin dryer, Electric rice cooker, Kitchen sink, Shoe rack, Trunk, suitcase, Bathtub, Air purifier, Video player, Bed, Coat rack, Display stand, Floor covering, Microwave, Vacuum cleaner, Table, Sofa, Door panel, Mat, Bamboo mat, Computer ,Main unit, Monitor, Printer, Humidifier, Bookstand, Piano, Aquarium, Carpet, PP bag, General waste bag

In [26]:
import onnxruntime as ort
import numpy as np
import cv2
import os

In [27]:
class_labels = {
0: "sewing machine",
1: "refrigerator",
2: "typewriter",
3: "Bookcase",
4: "Frame",
5: "lumber",
6: "TV",
7: "Audio",
8: "fax machine",
9: "chair",
10: "dish dryer",
11: "bicycle",
12: "arcade machine",
13: "TV stand",
14: "audio cabinet",
15: "Wardrobe",
16: "Display cabinet",
17: "Cupboard",
18: "Rice storage container",
19: "Stroller",
20: "Signboard",
21: "Washing machine",
22: "Speaker",
23: "Desk",
24: "Vanity table",
25: "Wall clock",
26: "Earthenware jar",
27: "Water tank",
28: "Air conditioner",
29: "Fan",
30: "Dining table",
31: "Small cabinet",
32: "Mannequin",
33: "Sink",
34: "Bedding items",
35: "Gas oven range",
36: "Heater",
37: "Cabinet",
38: "Drawer unit",
39: "Clothes drying rack",
40: "Toilet bowl",
41: "Spin dryer",
42: "Electric rice cooker",
43: "Kitchen sink",
44: "Shoe rack",
45: "Trunk, suitcase",
46: "Bathtub",
47: "Air purifier",
48: "Video player",
49: "Bed",
50: "Coat rack",
51: "Floor covering",
52: "Microwave",
53: "Vacuum cleaner",
54: "Table",
55: "Sofa",
56: "Door panel",
57: "Mat",
58: "Bamboo mat",
59: "Computer",
60: "Main unit",
61: "Monitor",
62: "Printer",
63: "Humidifier",
64: "Bookstand",
65: "Piano",
66: "Aquarium",
67: "Carpet",
68: "PP bag",
69: "General waste bag"
}

In [28]:
def iou(box1, box2):
    # 바운딩 박스 좌표 추출
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # 교차 영역 좌표 계산
    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)

    # 교차 영역 넓이 계산
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)

    # 각 바운딩 박스의 넓이 계산
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

    # IoU 계산
    iou_value = inter_area / (box1_area + box2_area - inter_area)
    return iou_value

In [29]:
def non_max_suppression(boxes, scores, iou_threshold=0.5):
    # 신뢰도를 기준으로 내림차순 정렬
    indices = np.argsort(scores)[::-1]
    selected_boxes = []

    while len(indices) > 0:
        # 신뢰도가 가장 높은 바운딩 박스를 선택
        current_box = boxes[indices[0]]
        selected_boxes.append(indices[0])

        # 나머지 바운딩 박스들과의 IoU 계산
        remaining_indices = []
        for i in indices[1:]:
            if iou(current_box, boxes[i]) < iou_threshold:
                remaining_indices.append(i)

        # 신뢰도가 높은 것만 남기기
        indices = remaining_indices

    return selected_boxes

In [33]:
def process_image(image_path, label):
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        return

    original_height, original_width = image.shape[:2]

    # 모델 입력 크기에 맞게 이미지 조정
    input_image = cv2.resize(image, (640, 640))  # YOLO-World 입력 크기에 맞게 조정
    input_image = input_image.transpose(2, 0, 1)  # 채널 순서 변경 (HWC -> CHW)
    input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가
    input_image = input_image.astype(np.float32) / 255.0  # 정규화

    # 입력 이름 확인
    input_name = ort_session.get_inputs()[0].name

    # 모델 실행
    outputs = ort_session.run(None, {input_name: input_image})

    # 출력 해석
    boxes = outputs[1][0]  # 바운딩 박스 좌표 (640x640 이미지 기준)
    scores = outputs[2][0]  # 신뢰도
    class_ids = outputs[3][0]  # 클래스 ID
    confidence_threshold = 0.2

    # # 신뢰도가 임계값을 넘는 객체만 필터링
    # filtered_indices = [i for i, score in enumerate(scores) if score > confidence_threshold]
    # filtered_boxes = boxes[filtered_indices]
    # filtered_scores = scores[filtered_indices]
    # filtered_class_ids = class_ids[filtered_indices]

    # # NMS 적용
    # selected_indices = non_max_suppression(filtered_boxes, filtered_scores, iou_threshold=0.5)

    # # 다수의 객체를 처리하는 반복문
    # for i in selected_indices:
    #     x1, y1, x2, y2 = filtered_boxes[i]  # 바운딩 박스 좌표 (640x640 기준)
        
    #     # 바운딩 박스 좌표를 원본 이미지 크기에 맞게 변환
    #     x1 = int(x1 / 640 * original_width)
    #     y1 = int(y1 / 640 * original_height)
    #     x2 = int(x2 / 640 * original_width)
    #     y2 = int(y2 / 640 * original_height)
        
    #     class_name = class_labels.get(int(filtered_class_ids[i]), 'Unknown')
    #     label = f"{class_name} {filtered_scores[i]:.2f}"

    #     # 색상 지정
    #     color = (0, 255, 0)  # 초록색
    #     cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 바운딩 박스 그리기
    #     cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # # 결과 이미지 표시
    # cv2.imshow('Result', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # 검출된 객체 정보 출력
    # print(f"이미지: {image_path}")
    # print("검출된 객체 정보:")
    for i in range(len(class_ids)):
        class_name = class_labels.get(int(class_ids[i]), 'Unknown')
        
        # print(f"클래스 ID: {class_name}, 신뢰도: {filtered_scores[i]:.2f}")
        if class_name == label:
            return True
            
    return False
    # print('-------------------')


In [34]:
def process_folder(folder_path):
    
    
    for root, dirs, files in os.walk(folder_path):
        sum_t = 0
        sum_f = 0
        count = 1
        # 현재 폴더의 이름을 가져온다
        current_folder_name = os.path.basename(root)
        
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(root, filename)
                
                if process_image(image_path, current_folder_name):
                    sum_t += 1
                else:
                    sum_f += 1
                
                # 폴더 이름과 파일 이름을 함께 출력
                print(f"Folder: {current_folder_name}, File: {filename}, Count: {count}")
                count += 1
    
        print(sum_t, sum_f)


In [36]:
ort_session = ort.InferenceSession("yolow-l.onnx")
folder_path = "images"  # 이미지가 저장된 폴더 경로 입력
process_folder(folder_path)

0 0
Folder: lumber, File: 13_X001_C162_1215_1.jpg, Count: 1
Folder: lumber, File: 13_X001_C808_0924_3.jpg, Count: 2
Folder: lumber, File: 13_X001_C808_0924_2.jpg, Count: 3
Folder: lumber, File: 13_X001_C162_1215_0.jpg, Count: 4
Folder: lumber, File: 13_X001_C162_1215_2.jpg, Count: 5
Folder: lumber, File: 13_X001_C808_0924_0.jpg, Count: 6
Folder: lumber, File: 13_X001_C808_0924_1.jpg, Count: 7
Folder: lumber, File: 13_X001_C162_1215_3.jpg, Count: 8
Folder: lumber, File: 13_X001_C808_0924_4.jpg, Count: 9
Folder: lumber, File: 13_X001_C162_1215_4.jpg, Count: 10
Folder: lumber, File: 13_X001_C211_1201_4.jpg, Count: 11
Folder: lumber, File: 13_X001_C162_1214_2.jpg, Count: 12
Folder: lumber, File: 13_X001_C162_1214_3.jpg, Count: 13
Folder: lumber, File: 13_X001_C162_1214_1.jpg, Count: 14
Folder: lumber, File: 13_X001_C189_1216_4.jpg, Count: 15
Folder: lumber, File: 13_X001_C162_1214_0.jpg, Count: 16
Folder: lumber, File: 13_X001_C211_1201_2.jpg, Count: 17
Folder: lumber, File: 13_X001_C189_1

In [ ]:

# ONNX 모델 로드
ort_session = ort.InferenceSession("yolow-l.onnx")

# 입력 이미지 준비
image_path = '148514863_1_1727006732_w480.jpg'
image = cv2.imread(image_path)
original_height, original_width = image.shape[:2]

# 모델 입력 크기에 맞게 이미지 조정
input_image = cv2.resize(image, (640, 640))  # YOLO-World 입력 크기에 맞게 조정
input_image = input_image.transpose(2, 0, 1)  # 채널 순서 변경 (HWC -> CHW)
input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가
input_image = input_image.astype(np.float32) / 255.0  # 정규화

# 입력 이름 확인
input_name = ort_session.get_inputs()[0].name

# 모델 실행
outputs = ort_session.run(None, {input_name: input_image})

# 출력 해석
boxes = outputs[1][0]  # 바운딩 박스 좌표 (640x640 이미지 기준)
scores = outputs[2][0]  # 신뢰도
class_ids = outputs[3][0]  # 클래스 ID
confidence_threshold = 0.2
class_name = np.empty(class_ids.shape, dtype=object)

# 신뢰도가 임계값을 넘는 객체만 필터링
filtered_indices = [i for i, score in enumerate(scores) if score > confidence_threshold]
filtered_boxes = boxes[filtered_indices]
filtered_scores = scores[filtered_indices]
filtered_class_ids = class_ids[filtered_indices]

# NMS 적용
selected_indices = non_max_suppression(filtered_boxes, filtered_scores, iou_threshold=0.5)

# 다수의 객체를 처리하는 반복문
for i in selected_indices:
    x1, y1, x2, y2 = filtered_boxes[i]  # 바운딩 박스 좌표 (640x640 기준)
    
    # 바운딩 박스 좌표를 원본 이미지 크기에 맞게 변환
    x1 = int(x1 / 640 * original_width)
    y1 = int(y1 / 640 * original_height)
    x2 = int(x2 / 640 * original_width)
    y2 = int(y2 / 640 * original_height)
    
    class_name[i] = class_labels.get(int(filtered_class_ids[i]), 'Unknown')
    label = f"{class_name[i]} {filtered_scores[i]:.2f}"

    # 색상 지정
    color = (0, 255, 0)  # 초록색
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 바운딩 박스 그리기
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
# 결과 이미지 표시
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 검출된 객체 정보 출력
print("검출된 객체 정보:")
for i in selected_indices:
    print(f"클래스 ID: {class_name[i]}, 신뢰도: {filtered_scores[i]:.2f}")

In [ ]:

# ONNX 모델 로드
ort_session = ort.InferenceSession("yolow-l.onnx")

# 이미지가 저장된 폴더 경로
folder_path = 'image'

# 폴더 내 모든 파일 반복
for filename in os.listdir(folder_path):
    # 파일의 확장자가 이미지 형식인지 확인
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)  # 전체 이미지 경로

        # 입력 이미지 준비
        image = cv2.imread(image_path)
        original_height, original_width = image.shape[:2]

        # 모델 입력 크기에 맞게 이미지 조정
        input_image = cv2.resize(image, (640, 640))  # YOLO-World 입력 크기에 맞게 조정
        input_image = input_image.transpose(2, 0, 1)  # 채널 순서 변경 (HWC -> CHW)
        input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가
        input_image = input_image.astype(np.float32) / 255.0  # 정규화

        # 입력 이름 확인
        input_name = ort_session.get_inputs()[0].name

        # 모델 실행
        outputs = ort_session.run(None, {input_name: input_image})

        # 출력 해석
        boxes = outputs[1][0]  # 바운딩 박스 좌표 (640x640 이미지 기준)
        scores = outputs[2][0]  # 신뢰도
        class_ids = outputs[3][0]  # 클래스 ID
        confidence_threshold = 0.1

        # 다수의 객체를 처리하는 반복문
        for i in range(len(scores)):
            score = scores[i]
            if score > confidence_threshold:  # 신뢰도가 임계값보다 높은 객체만 처리
                x1, y1, x2, y2 = boxes[i]  # 바운딩 박스 좌표 (640x640 기준)

                # 바운딩 박스 좌표를 원본 이미지 크기에 맞게 변환
                x1 = int(x1 / 640 * original_width)
                y1 = int(y1 / 640 * original_height)
                x2 = int(x2 / 640 * original_width)
                y2 = int(y2 / 640 * original_height)

                class_id = int(class_ids[i])  # 클래스 ID
                label = f"Class {class_id} {score:.2f}"

                # 색상 지정
                color = (0, 255, 0)  # 초록색
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 바운딩 박스 그리기
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # 결과 이미지 표시
        cv2.imshow('Result', image)
        cv2.waitKey(0)  # 결과 창에서 키 입력을 기다림
        cv2.destroyAllWindows()

        # 검출된 객체 정보 출력
        print(f"파일: {filename}")
        print("검출된 객체 정보:")
        for i in range(len(scores)):
            score = scores[i]
            if score > confidence_threshold:
                class_id = int(class_ids[i])
                print(f"클래스 ID: {class_id}, 신뢰도: {score:.2f}")

